<a href="https://colab.research.google.com/github/galonpy/youtube_llm/blob/main/BERTopic_Better_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
#!pip install bertopic

Understanding the YouTube Dislikes Dataset from Kaggle
https://www.kaggle.com/datasets/dmitrynikolaev/youtube-dislikes-dataset

Youtube announced the decision to hide the number of dislikes from users around November 2021. However, the official YouTube Data API allowed you to get information about dislikes until December 13, 2021.

This dataset contains the latest possible information about dislikes, which was collected just before December 13

In [6]:
  import pandas as pd
  from bertopic import BERTopic

In [13]:
df = pd.read_parquet('/content/drive/MyDrive/english_to_spanish.parquet')

In [32]:
df.head(5)

,english,spanish,citation
0,Go.,Ve.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Vete.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Vaya.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Váyase.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Hola.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


The above median dataset has some the more disliked videos

In [16]:
english_sentences = df['english'].tolist()
spanish_sentences = df['spanish'].tolist()

In [19]:
sample_english = english_sentences[:500]

In [17]:
#from bertopic.representation import KeyBERTInspired

# Fine-tune your topic representations
representation_model = BERTopic()
topic_model = BERTopic(representation_model=representation_model)

In [21]:
topics, probs = topic_model.fit_transform(sample_english)

In [22]:
topic_labels = topic_model.generate_topic_labels(nr_words=3,topic_prefix=False, separator = " - ")
topic_model.set_topic_labels(topic_labels)

In [23]:
topic_model.get_topic_info()[1:10]

,Topic,Count,Name,CustomName,Representation,Representative_Docs
1,0,57,0_out_up_get_stop,out - up - get,"[out, up, get, stop, shut, wake, leave, us, ke...","[Get out., Get out!, Get out.]"
2,1,50,1_tell_we_try_got,tell - we - try,"[tell, we, try, got, know, it, me, did, failed...","[Tell me., Tell me., We try.]"
3,2,47,2_im_its_fat_am,im - its - fat,"[im, its, fat, am, old, thin, bald, rich, okay...","[I am old., Am I fat?, Am I fat?]"
4,3,33,3_tom_call_find_ask,tom - call - find,"[tom, call, find, ask, kiss, them, save, phone...","[Call Tom., Call Tom., I'm Tom.]"
5,4,27,4_shoot_attack_thanks_fire,shoot - attack - thanks,"[shoot, attack, thanks, fire, welcome, awesome...","[Shoot!, Shoot!, Shoot!]"
6,5,25,5_ran_moved_hid_fled,ran - moved - hid,"[ran, moved, hid, fled, he, left, came, who, s...","[I ran., He ran., He ran.]"
7,6,25,6_away_go_lets_off,away - go - lets,"[away, go, lets, off, goodbye, back, cool, , , ]","[Go away!, Go away!, Go away.]"
8,7,23,7_push_it_hold_lock,push - it - hold,"[push, it, hold, lock, him, fold, drop, cuff, ...","[Push it., Push it., Push it.]"
9,8,20,8_be_good_fun_have,be - good - fun,"[be, good, fun, have, quiet, nice, man, kind, ...","[Be good., Be good., Be good.]"


In [24]:
topic_model.visualize_barchart(top_n_topics=15,custom_labels=True)

In [25]:
topic_model.visualize_heatmap(top_n_topics=10)

In [26]:
similar_topics, similarity = topic_model.find_topics("politics", top_n=3); similar_topics

[-1, 14, 1]

In [27]:
#topic_model.get_topic(176)

In [29]:
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
#docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(sample_english, show_progress_bar=False)

# Train BERTopic
topic_model = BERTopic().fit(sample_english, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(sample_english, embeddings=embeddings,topics=list(range(15)),custom_labels=True,height=600)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
